In [ ]:
%%time

import os
from sales_forecasting_model import *
from skt.vault_utils import get_secrets

user = os.environ['USER']
proxy = get_secrets("proxy")["proxy"]
os.environ['HTTP_PROXY'] = proxy
os.environ['HTTPS_PROXY'] = proxy
os.environ["TOKENIZERS_PARALLELISM"] = "false"

model_type = 'gru'

cases = [
    [False, True], [True, True]
]

for item in range(10, 51):
    for use_bert, add_total in cases:
        dict_args = {
            'auto_lr_find': True,
            'gradient_clip_val': 1.0,
            'accelerator': 'dp',
            'strategy': 'dp',
            'amp_backend': 'apex', 
            'amp_level': 'O2',
            'accumulate_grad_batches': 8,

            'seed': 1234, 
            'workers': 2,
            'max_length': 32,
            'batch_size': 32,
            'max_epochs': 100,
            'lr': 0.002, 

            'gpus': -1,
            #'gpu_list': '2;3', 
            'train_file': f'sales_data/train_items/Item{item}.json',
            'val_file': f'sales_data/val_items/Item{item}.json',
            'test_file': f'sales_data/test_items/Item{item}.json',

            'item': item,
            'transformer_path': 'sales_data/power_transformer.pkl',
            'model_type': model_type, 
            'input_dim': 10,
            'output_dim': 10,
            'hidden_dim': 20,
            'n_seq': 7,
            'drop_prob': 0.2,
            'use_bert': use_bert,
            'add_total': add_total
        }
        train(dict_args)
        #case_name = f'item{item}_{model_type}_bert{use_bert}_total{add_total}_{dict_args["lr"]}_{dict_args["seed"]}'
        #evaluate_models(f'/home/{user}/logs_{case_name}')